In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger


Using TensorFlow backend.


In [2]:
traindata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_train.csv', index_col=None)
testdata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_test.csv', index_col=None)

traindata.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",land,wrong_fragment,urgent,"""hot""",...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,S1,278,7958,0,0,0,0,...,255,1.00,0.00,0.33,0.03,0.33,0.0,0.0,0.0,normal
1,0,tcp,http,SF,241,851,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.00,0.0,0.0,0.0,normal
2,0,udp,private,SF,105,146,0,0,0,0,...,239,0.94,0.01,0.01,0.00,0.00,0.0,0.0,0.0,normal
3,0,tcp,http,SF,367,5199,0,0,0,0,...,255,1.00,0.00,0.01,0.05,0.00,0.0,0.0,0.0,normal
4,21263,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.66,1.00,0.00,0.00,0.0,1.0,1.0,probe


In [3]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in traindata.columns:
    #print df[c].dtype
    if traindata[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        traindata[c] = encodings[c].fit_transform(traindata[c])

In [4]:
encodings1 = dict()
for c in testdata.columns:
    #print df[c].dtype
    if testdata[c].dtype == "object":
        encodings1[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings1[c]
        testdata[c] = encodings1[c].fit_transform(testdata[c])

In [5]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]
Y.head()

0    1
1    1
2    1
3    1
4    2
Name: label, dtype: int32

In [6]:
X=X.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1) 
T=T.drop(['land','su_attempted','num_outbound_cmds','is_host_login','urgent','num_failed_logins','su_attempted','num_file_creations','num_shells','srv_diff_host_rate'], axis = 1) 

In [7]:
T.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",wrong_fragment,"""hot""",logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,3047,2,26,5,146,105,0,0,0,0,...,255,1,0.00,0.69,0.96,0.00,0.0,0.0,0.0,0.0
1,0,1,15,5,202,1923,0,0,1,0,...,21,255,1.00,0.00,0.05,0.02,0.0,0.0,0.0,0.0
2,0,2,6,5,44,72,0,0,0,0,...,255,254,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
3,0,1,27,3,0,0,0,0,0,0,...,255,14,0.05,0.08,0.00,0.00,1.0,1.0,0.0,0.0
4,0,1,27,2,0,0,0,0,0,0,...,255,1,0.00,0.82,1.00,0.00,0.0,0.0,1.0,1.0


In [8]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
print(trainX[0:5,:])

[[0.000e+00 1.255e-04 2.636e-03 5.021e-04 3.489e-02 9.989e-01 0.000e+00
  0.000e+00 1.255e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.255e-04 2.510e-04 1.255e-04 6.276e-05 0.000e+00 0.000e+00 1.255e-04
  0.000e+00 3.766e-04 3.201e-02 1.255e-04 0.000e+00 4.142e-05 3.766e-06
  4.142e-05 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.075e-03 2.257e-02 5.375e-03 2.591e-01 9.148e-01 0.000e+00
  0.000e+00 1.075e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.827e-02 3.762e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.075e-03
  0.000e+00 1.365e-01 2.741e-01 1.075e-03 0.000e+00 1.075e-05 3.225e-05
  0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 5.062e-03 1.012e-01 1.265e-02 2.657e-01 3.695e-01 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  7.592e-03 5.062e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.696e-03
  1.696e-03 6.454e-01 6.049e-01 2.379e-03 2.531e-05 2.531e-05 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000

In [9]:
scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

In [10]:
y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1)

In [11]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


batch_size = 64

In [14]:
# 1. define the network
model = Sequential()
model.add(SimpleRNN(1024,input_dim=32, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(SimpleRNN(1024,return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(5))
model.add(Activation('softmax'))


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(1024, return_sequences=True, input_shape=(None, 32))`
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# 1. define the network
model = Sequential()
model.add(LSTM(16,input_dim=32, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(LSTM(16, return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(5))
model.add(Activation('softmax'))

C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, return_sequences=True, input_shape=(None, 32))`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# try using different optimizers and different optimizer configs
import time
start = time.time()
import tensorflow as tf
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
#csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test))#,callbacks=[checkpointer,csv_logger])
#model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")

#loss, accuracy = model.evaluate(X_test, y_test)
#print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
#y_pred = model.predict_classes(X_test)
#np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', np.transpose([y_test1,y_pred]), fmt='%01d')
end = time.time()
print('exec time = '+str(end-start))


C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Instructions for updating:
Use tf.cast instead.
Train on 808 samples, validate on 203 samples
Epoch 1/1000
808/808 [==============================] - 5s 6ms/step - loss: 0.6857 - accuracy: 0.7958 - precision_1: 0.7201 - recall_1: 0.4524 - val_loss: 0.4974 - val_accuracy: 0.8276 - val_precision_1: 0.8470 - val_recall_1: 0.6923
Epoch 2/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.4545 - accuracy: 0.8292 - precision_1: 0.8558 - recall_1: 0.7214 - val_loss: 0.5354 - val_accuracy: 0.8325 - val_precision_1: 0.8601 - val_recall_1: 0.7437
Epoch 3/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.3966 - accuracy: 0.8564 - precision_1: 0.8615 - recall_1: 0.7627 - val_loss: 0.4531 - val_accuracy: 0.8128 - val_precision_1: 0.8616 - val_recall_1: 0.7758
Epoch 4/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.3737 - accuracy: 0.8688 - precision_1: 0.8643 - recall_1: 0.7834 - val_loss: 0.4970 - val_accuracy: 0.8177 - val_precision_1: 

808/808 [==============================] - 2s 2ms/step - loss: 0.2411 - accuracy: 0.9121 - precision_1: 0.9022 - recall_1: 0.8690 - val_loss: 0.3283 - val_accuracy: 0.8966 - val_precision_1: 0.9025 - val_recall_1: 0.8696
Epoch 35/1000
808/808 [==============================] - 2s 3ms/step - loss: 0.2272 - accuracy: 0.9208 - precision_1: 0.9026 - recall_1: 0.8700 - val_loss: 0.3106 - val_accuracy: 0.9163 - val_precision_1: 0.9030 - val_recall_1: 0.8708
Epoch 36/1000
808/808 [==============================] - 2s 3ms/step - loss: 0.2100 - accuracy: 0.9319 - precision_1: 0.9035 - recall_1: 0.8716 - val_loss: 0.2733 - val_accuracy: 0.9458 - val_precision_1: 0.9040 - val_recall_1: 0.8723
Epoch 37/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.2002 - accuracy: 0.9381 - precision_1: 0.9046 - recall_1: 0.8731 - val_loss: 0.2992 - val_accuracy: 0.9212 - val_precision_1: 0.9049 - val_recall_1: 0.8739
Epoch 38/1000
808/808 [==============================] - 2s 3ms/step - los

Epoch 69/1000
808/808 [==============================] - 2s 3ms/step - loss: 0.1865 - accuracy: 0.9369 - precision_1: 0.9201 - recall_1: 0.8991 - val_loss: 0.3320 - val_accuracy: 0.9064 - val_precision_1: 0.9201 - val_recall_1: 0.8993
Epoch 70/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1570 - accuracy: 0.9468 - precision_1: 0.9203 - recall_1: 0.8995 - val_loss: 0.2276 - val_accuracy: 0.9409 - val_precision_1: 0.9205 - val_recall_1: 0.8998
Epoch 71/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1628 - accuracy: 0.9394 - precision_1: 0.9207 - recall_1: 0.9001 - val_loss: 0.2115 - val_accuracy: 0.9606 - val_precision_1: 0.9209 - val_recall_1: 0.9004
Epoch 72/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1497 - accuracy: 0.9443 - precision_1: 0.9211 - recall_1: 0.9007 - val_loss: 0.2103 - val_accuracy: 0.9557 - val_precision_1: 0.9213 - val_recall_1: 0.9010
Epoch 73/1000
808/808 [==============================] - 2s 

Epoch 104/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1329 - accuracy: 0.9517 - precision_1: 0.9286 - recall_1: 0.9126 - val_loss: 0.2181 - val_accuracy: 0.9507 - val_precision_1: 0.9287 - val_recall_1: 0.9127
Epoch 105/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1489 - accuracy: 0.9480 - precision_1: 0.9289 - recall_1: 0.9129 - val_loss: 0.2017 - val_accuracy: 0.9557 - val_precision_1: 0.9290 - val_recall_1: 0.9130
Epoch 106/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1310 - accuracy: 0.9554 - precision_1: 0.9291 - recall_1: 0.9132 - val_loss: 0.2115 - val_accuracy: 0.9507 - val_precision_1: 0.9292 - val_recall_1: 0.9134
Epoch 107/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.1243 - accuracy: 0.9530 - precision_1: 0.9294 - recall_1: 0.9136 - val_loss: 0.1962 - val_accuracy: 0.9557 - val_precision_1: 0.9295 - val_recall_1: 0.9137
Epoch 108/1000
808/808 [==============================] 

...

Epoch 383/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0630 - accuracy: 0.9777 - precision_1: 0.9558 - recall_1: 0.9490 - val_loss: 0.2230 - val_accuracy: 0.9507 - val_precision_1: 0.9558 - val_recall_1: 0.9490
Epoch 384/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0604 - accuracy: 0.9765 - precision_1: 0.9558 - recall_1: 0.9491 - val_loss: 0.2092 - val_accuracy: 0.9704 - val_precision_1: 0.9559 - val_recall_1: 0.9491
Epoch 385/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0649 - accuracy: 0.9678 - precision_1: 0.9559 - recall_1: 0.9491 - val_loss: 0.1998 - val_accuracy: 0.9409 - val_precision_1: 0.9559 - val_recall_1: 0.9491
Epoch 386/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0865 - accuracy: 0.9703 - precision_1: 0.9559 - recall_1: 0.9492 - val_loss: 0.2664 - val_accuracy: 0.9409 - val_precision_1: 0.9559 - val_recall_1: 0.9492
Epoch 387/1000
808/808 [==============================] 

Epoch 418/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0517 - accuracy: 0.9851 - precision_1: 0.9572 - recall_1: 0.9508 - val_loss: 0.1682 - val_accuracy: 0.9606 - val_precision_1: 0.9572 - val_recall_1: 0.9508
Epoch 419/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0535 - accuracy: 0.9802 - precision_1: 0.9572 - recall_1: 0.9508 - val_loss: 0.1727 - val_accuracy: 0.9557 - val_precision_1: 0.9572 - val_recall_1: 0.9508
Epoch 420/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0698 - accuracy: 0.9728 - precision_1: 0.9573 - recall_1: 0.9509 - val_loss: 0.1703 - val_accuracy: 0.9606 - val_precision_1: 0.9573 - val_recall_1: 0.9509
Epoch 421/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0619 - accuracy: 0.9765 - precision_1: 0.9573 - recall_1: 0.9509 - val_loss: 0.1797 - val_accuracy: 0.9557 - val_precision_1: 0.9573 - val_recall_1: 0.9509
Epoch 422/1000
808/808 [==============================] 

808/808 [==============================] - 2s 3ms/step - loss: 0.0526 - accuracy: 0.9827 - precision_1: 0.9586 - recall_1: 0.9525 - val_loss: 0.1770 - val_accuracy: 0.9507 - val_precision_1: 0.9586 - val_recall_1: 0.9525
Epoch 453/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0462 - accuracy: 0.9889 - precision_1: 0.9586 - recall_1: 0.9526 - val_loss: 0.1831 - val_accuracy: 0.9507 - val_precision_1: 0.9586 - val_recall_1: 0.9526
Epoch 454/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0497 - accuracy: 0.9839 - precision_1: 0.9587 - recall_1: 0.9526 - val_loss: 0.2123 - val_accuracy: 0.9507 - val_precision_1: 0.9587 - val_recall_1: 0.9526
Epoch 455/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0471 - accuracy: 0.9814 - precision_1: 0.9587 - recall_1: 0.9527 - val_loss: 0.2133 - val_accuracy: 0.9360 - val_precision_1: 0.9587 - val_recall_1: 0.9527
Epoch 456/1000
808/808 [==============================] - 2s 2ms/step -

Epoch 487/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0587 - accuracy: 0.9740 - precision_1: 0.9596 - recall_1: 0.9539 - val_loss: 0.1849 - val_accuracy: 0.9507 - val_precision_1: 0.9597 - val_recall_1: 0.9539
Epoch 488/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0521 - accuracy: 0.9876 - precision_1: 0.9597 - recall_1: 0.9539 - val_loss: 0.1595 - val_accuracy: 0.9655 - val_precision_1: 0.9597 - val_recall_1: 0.9539
Epoch 489/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0692 - accuracy: 0.9790 - precision_1: 0.9597 - recall_1: 0.9539 - val_loss: 0.1953 - val_accuracy: 0.9606 - val_precision_1: 0.9597 - val_recall_1: 0.9540
Epoch 490/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0552 - accuracy: 0.9851 - precision_1: 0.9598 - recall_1: 0.9540 - val_loss: 0.2154 - val_accuracy: 0.9360 - val_precision_1: 0.9598 - val_recall_1: 0.9540
Epoch 491/1000
808/808 [==============================] 

Epoch 522/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0468 - accuracy: 0.9827 - precision_1: 0.9609 - recall_1: 0.9553 - val_loss: 0.1772 - val_accuracy: 0.9606 - val_precision_1: 0.9609 - val_recall_1: 0.9554
Epoch 523/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0482 - accuracy: 0.9839 - precision_1: 0.9609 - recall_1: 0.9554 - val_loss: 0.2593 - val_accuracy: 0.9409 - val_precision_1: 0.9609 - val_recall_1: 0.9554
Epoch 524/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0523 - accuracy: 0.9864 - precision_1: 0.9609 - recall_1: 0.9554 - val_loss: 0.1783 - val_accuracy: 0.9557 - val_precision_1: 0.9609 - val_recall_1: 0.9554
Epoch 525/1000
808/808 [==============================] - 2s 2ms/step - loss: 0.0487 - accuracy: 0.9765 - precision_1: 0.9610 - recall_1: 0.9555 - val_loss: 0.1551 - val_accuracy: 0.9655 - val_precision_1: 0.9610 - val_recall_1: 0.9555
Epoch 526/1000
808/808 [==============================] 